## Create Distance Matrix

In [1]:
# Install modules
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install prettytable
!{sys.executable} -m pip install ipyparallel

import random
import copy
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
from prettytable import PrettyTable
import ipyparallel as ipp
from ipyparallel.util import interactive

# Create distance matrix (excluding distance from each point to itself)
burma_tree = ET.parse('burma14.xml')
burma_root = burma_tree.getroot()
brazil_tree = ET.parse('brazil58.xml')
brazil_root = brazil_tree.getroot()

d = []
edge_num = 14
country = "burma" #brazil, burma

if country == "brazil":
    edge_num = 58
    
    for vertex in brazil_root.iter('vertex'):
        temp = [0]*edge_num
        for edge in vertex.iter('edge'):
            temp[int(edge.text)] = (float(edge.get('cost')))
        d.append(temp)
else:
    for vertex in burma_root.iter('vertex'):
        temp = [0]*edge_num
        for edge in vertex.iter('edge'):
            temp[int(edge.text)] = (float(edge.get('cost')))
        d.append(temp)

    
d = np.array(d)

# Methods

In [2]:
def remove_current_city(city, H):
    """
    Removes current node from hueristic matrix

    :param city: the current node
    :param H: hueristic matrix
    :return: new heuristic matrix
    """ 
    
    for index, x in np.ndenumerate(H):
        if city == index[1] or x == 0:
            H[index] = 0
        else:
            H[index] = round(1/d[index[0]][index[1]],4)
    return H

def create_probabilities(city, H, T, alpha, beta):
    """
    Create probability matrix using heuristic and pheromones

    :param city: the current node
    :param H: hueristic matrix
    :param T: pheromone matrix
    :param alpha: weighting of pheromones
    :param beta: weighting of heuristics
    :return: probability matrix
    """ 
    
    P = np.zeros(edge_num)
    den = 0

    # for each value in column of current city, plug number into formula x^alpha * y^beta
    
    # Numerators
    N = np.array(T[city, :] **alpha * H[city, :] **beta)
    
    # Denominator (sum of all numerators)
    den = np.sum(N)
    
    # Probabilities
    P = N/den
    
    return P

def find_next_city(P, city):
    """
    Determines the next node for an ant to go to

    :param P: probability matrix
    :param city: current node
    :return: next node
    """ 
    rand = random.random()
    CP = 0
    for i in range(edge_num):
        CP += P[i]
        if CP >= rand:
            return i
        
def create_ants(num, start, H, T, alpha, beta):
    """
    Create set of ants

    :param num: number of ants to create
    :param start: starting node for ants
    :param H: heuristics matrix
    :param T: pheromones matrix
    :param alpha: weighting of pheromones
    :param beta: weighting of heuristics
    :return: list of ant objects
    """

    # Initial probabilities
    P = create_probabilities(start, copy.deepcopy(H), copy.deepcopy(T), alpha, beta)

    # Ant dictionary
    ant_dict = {
    "H": copy.deepcopy(H),
    "T": copy.deepcopy(T),
    "P": copy.deepcopy(P),
    "Length": 0,
    "Path": [0]
    }

    # Create ants
    Ants = np.array([copy.deepcopy(ant_dict) for i in range(num + 1)])

    return Ants

def run_ants(start, Ants, alpha, beta):
    """
    Run the ants over the nodes storing a path and length for each

    :param start: starting node
    :param Ants: list of ant objects
    :param alpha: weighting of pheromones
    :param beta: weighting of heuristics
    :return: list of ant objects
    """ 
    
    def run_singular_ant(ant):
        """
        Vectorized function to run on every ant
        
        :param ant: ant to run function on
        :return: alters ant values
        """
        
        # Start city
        current_city = start
        
        # Go to every node
        for i in range(edge_num - 1):
            
            # Remove this city from heuristics
            ant["H"] = remove_current_city(current_city, ant["H"])

            # Create probabilities
            ant["P"] = create_probabilities(current_city, ant["H"], ant["T"], alpha, beta)

            # Get next city to go to
            temp = current_city
            current_city = find_next_city(ant["P"], temp)

            # Update length
            ant["Length"] += d[temp][current_city]

            # Update path
            ant["Path"].append(current_city)

            # Add the length to get back to starting node
            
 
        ant["Length"] += d[current_city][start]
        ant["H"] = np.array([[ round(1/d[i][j],4) if i != j else 0 for j in range(edge_num)] for i in range(edge_num)])
    
    # Vectorize function
    v_run_ant = np.vectorize(run_singular_ant)
    v_run_ant(Ants)
    
    # remove first ant (dummy first ant for vectorizing to establish types by running first element twice https://numpy.org/doc/stable/reference/generated/numpy.vectorize.html)
    Ants = Ants[1 :]
    
    return Ants


def evapourate(T, e):
    """
    Evapourate pheromones

    :param T: pheromones matrix
    :param e: evapouration rate
    :return: new pheromones matrix
    """ 
            
    return np.multiply(T, (1-e))
    

def pheromone_update(Ants, Q, T, h_function):
    """
    Updates pheromones for acs approach

    :param Ants: list of ant objects
    :param Q: hyper parameter to scale pheromone deposits
    :param T: pheromones matrix
    :param h_function: local heuristic function ("Qd" | (float))
    :return: new pheromones matrix
    """
    
    for ant in Ants:
        
        
        # Determine the amount to deposit (depending on heuristic function)
        if h_function == "Qd":
            delta = Q/ant["Length"]
        else:
            delta = h_function
        
        # Deposit on each path taken
        x = [[item] for item in ant["Path"][1::]]
        y = [[item] for item in ant["Path"][:-1]]
        
        indices = np.hstack([y, x])
        
        np.add.at(T, (indices[:, 0], indices[:, 1]), delta)
            
    return T

def pheromone_update_mmas(Ants, Q, T, global_best, mmas_type, upper_limit, lower_limit, h_function):
    """
    Updates pheromones for acs approach

    :param Ants: list of ant objects
    :param Q: hyper parameter to scale pheromone deposits
    :param T: pheromones matrix
    :param global_best: global best solution
    :param mmas_type: type of mmas ("global" or "local")
    :param upper_limit: upper pheromone limit
    :param lower_limit: lower pheromone limit
    :param h_function: local heuristic function ("Qd" | (float))
    :return: new pheromones matrix
    """
    
    # Check mmas type (global or local)
    if mmas_type == "global":
        # Global
        best_length = global_best["Length"]
    else:
        # Local
        best_length = find_best(Ants)["Length"]
        
        
    # Loop through every ant
    for ant in Ants:
        
        # If better than or equal to the best length then deposit
        if ant["Length"] <= best_length:
        
            # Determine the amount to deposit (depending on heuristic function)
            if h_function == "Qd":
                delta = Q/ant["Length"]
            else:
                delta = h_function

            # Limit pheromones
            if delta > upper_limit: delta = upper_limit
            elif delta < lower_limit: delta = lower_limit


            # Deposit on each path taken
            x = [[item] for item in ant["Path"][1::]]
            y = [[item] for item in ant["Path"][:-1]]

            indices = np.hstack([y, x])

            np.add.at(T, (indices[:, 0], indices[:, 1]), delta)
            
    return T

def pheromone_update_elitist(Ants, Q, T, global_best, h_function):
    """
    Updates pheromones for elitist approach (global best always deposited even if not visited)

    :param Ants: list of ant objects
    :param Q: hyper parameter to scale pheromones
    :param T: pheromones matrix
    :param global_best: global best path and length
    :param h_function: local heuristic function ("Qd" | (float))
    :return: void
    """ 
    
    # Find local best
    local_best = find_best(Ants)
    
    # If local is worse than global (global hasn't been visited) then deposit global 
    if local_best["Length"] < global_best["Length"]:
        
        # Determine the amount to deposit (depending on heuristic function)
        if h_function == "Qd":
            delta = Q/ant["Length"]
        else:
            delta = h_function
        
        # Deposit on best global path
        x = [[item] for item in global_best["Path"][1::]]
        y = [[item] for item in global_best["Path"][:-1]]
        indices = np.hstack([y, x])

        np.add.at(T, (indices[:, 0], indices[:, 1]), delta)
    
    # Loop through every ant
    for ant in Ants:
        
        # Determine the amount to deposit
        delta = Q/ant["Length"]

        # Deposit on each path taken
        x = [[item] for item in ant["Path"][1::]]
        y = [[item] for item in ant["Path"][:-1]]
        indices = np.hstack([y, x])

        np.add.at(T, (indices[:, 0], indices[:, 1]), delta)
        
    return T

def find_best(Ants):
    """
    Find the best solution

    :param Ants: list of ant objects
    :return: best solution
    """ 
    
    # Set best to first ant
    smallest = Ants[0]["Length"]
    path = Ants[0]["Path"]
    
    # Find the smallest length
    for ant in Ants:
        if ant["Length"] < smallest:
            smallest = ant["Length"]
            path = ant["Path"]

    return {"Length": smallest, "Path": path}

## Main ACO Loop

In [3]:
# Main loop
def create_and_run_ants(num_ants, num_iters, starting_city, alpha, beta, Q, e, T, H, h_function, variation):
    """
    Create and run a set of ants

    :param num_ants: number of ants to create
    :param num_ants: number of iterations to perform
    :param starting_city: starting node
    :param alpha: weighting of pheromones
    :param beta: weighting of heuristics
    :param Q: hyper parameter to scale pheromone deposits
    :param e: evapouration rate
    :param T: pheromones matrix
    :param H: heuristics matrix
    :param variation: aco variation (
    {
        "approach": "as" | "mmas" | "elitist",
        "upper": int,
        "lower": int,
        "type": "global" | "local"
    })

    :return: list of ant objects
    """ 

    
    # Loop for number of iterations
    for i in range(num_iters):
        
        # Create and run ants
        Ants = create_ants(num_ants, starting_city, H, T, alpha, beta)
        run_ants(starting_city, Ants, alpha, beta)
    
        # Keep track of global best
        local_best = find_best(Ants)
        if i == 0 or (local_best["Length"] < global_best["Length"]):
            global_best = local_best
        
        # Check variation
        if variation["approach"] == "mmas":
            T = pheromone_update_mmas(Ants, Q, T, global_best, variation["type"], variation["upper"], variation["lower"], h_function)
        elif variation["approach"] == "elitist":
            T = pheromone_update_elitist(Ants, Q, T, global_best, h_function)
        else:
            T = pheromone_update(Ants, Q, T, h_function)
        
        # Evapourate
        T = evapourate(T, e)
        

        
    return global_best

## Multiple ACOs

In [4]:
def run_ACOs(num_ants, num_iterations, num_clusters, starting_city, alpha, beta, Q, e, h_function, variation):
    """
    Run ACO "num_iterations" times with "num_clusters" clusters and print best solution from each cluster
    
    :param num_ants: number of ants to create
    :param num_ants: number of iterations to perform
    :param num_clusters: number of clusters to run
    :param starting_city: starting node
    :param alpha: weighting of pheromones
    :param beta: weighting of heuristics
    :param Q: hyper parameter to scale pheromone deposits
    :param e: evapouration rate
    :param T: pheromones matrix
    :param H: heuristics matrix
    :param h_function: local heuristic function ("Qd" | (float))
    :param variation: aco variation (
    {
        "approach": "as" | "mmas" | "elitist",
        "upper": int,
        "lower": int,
        "type": "global" | "local"
    })
    """
    
    # Start n clusters
    with ipp.Cluster(n=num_clusters) as rc:

        view = rc[:]

        # Define variables in correct space
        view.execute("import copy, random")
        view.execute("import numpy as np")
        view['create_probabilities'] = create_probabilities
        view['create_ants'] = create_ants
        view['remove_current_city'] = remove_current_city
        view['find_next_city'] = find_next_city
        view['edge_num'] = edge_num
        view['run_ants'] = run_ants
        view['d'] = d
        view['create_and_run_ants'] = create_and_run_ants
        view['find_best'] = find_best
        view['pheromone_update'] = pheromone_update
        view['pheromone_update_mmas'] = pheromone_update_mmas
        view['pheromone_update_elitist'] = pheromone_update_elitist
        view['evapourate'] = evapourate

        view['num_ants'] = num_ants
        view['num_iterations'] = num_iterations
        view['starting_city'] = starting_city
        view['alpha'] = alpha
        view['beta'] = beta
        view['Q'] = Q
        view['e'] = e
        view['h_function'] = h_function
        view['variation'] = variation

        # Initial Heuristic Matrix
        H = np.array([[ round(1/d[i][j],4) if i != j else 0 for j in range(edge_num)] for i in range(edge_num)])

        # Initial Pheromone Matrix
        if variation["approach"] == "mmas":
            # Start with upper limit pheromones
            T = np.array([[variation["upper"] for j in range(edge_num)] for i in range(edge_num)])
        else:
            # Start with random pheromones
            T = np.array([[random.random() for j in range(edge_num)] for i in range(edge_num)])

        # Run async task to create and run ants
        asyncresult = view.apply_async(create_and_run_ants, num_ants, num_iterations, starting_city, alpha, beta, Q, e, T, H, h_function, variation)
        asyncresult.wait_interactive()
        result = asyncresult.get()

    # Return result
    return result
            

## Execute ACOs

In [5]:
def run_experiments(exp):
    # Base Parameters <----------- (DO NOT CHANGE)

    params = {
        "num_ants":  1000, # (int)
        "num_iterations": 10, # (int)
        "num_clusters": 10, # (int)
        "starting_city": 0, # (int)
        "alpha": 0.5, # (float)
        "beta": 1, # (float)
        "Q": 1, # (int)
        "e": 0.5, # (float)
        "h_function": "Qd", # ("Qd" | float)
        "variation": {
            "approach": "as", # ("as" | "mmas" | "elitist")
            "upper": 1, # (float)
            "lower": 0, # (float)
            "type": "global", #("global" | "local")
        },
    }


    # Create output table
    table_fields = ['Value', 'Avg Length', 'Best Length', 'Best Path']
    output_table = PrettyTable(table_fields)
    output_table.padding_width = 1

    for value in exp["values"]:
        # Set param to next value
        params[exp["name"]] = value

        # If num_ants is changed then num_iterations must also be changed to ensure 10,000 fitness evaluations
        if exp["name"] == "num_ants":
            params["num_iterations"] = round(10_000 / value)

        # Run ACO with experimental param
        result = run_ACOs(params["num_ants"],
            params["num_iterations"],
            params["num_clusters"],
            params["starting_city"],
            params["alpha"],
            params["beta"],
            params["Q"],
            params["e"],
            params["h_function"],
            params["variation"])


        # Record best solution
        best = find_best(result)

        # Record mean length
        df = pd.DataFrame.from_dict(result)
        avg = df['Length'].sum()
        avg /= params["num_clusters"]


        # Add row to table
        output_table.add_row([value, avg, best["Length"], best["Path"]])

    # Set table name
    output_table.title = exp["name"]

    # Print and save to txt
    print(output_table)
    data = output_table.get_string()
    with open(str(exp["name"] + ".txt"), 'w') as f:
        f.write(data)



## Experiment with Parameters

In [ ]:
# --------------- ONLY CHANGE THESE VALUES --------------- # 

# Experiment Parameters
exp = {
    "name": "e", # name of parameter
    "values": [0.1, 0.3, 0.5, 0.7, 0.9] # list of values to try
}

"""
Possible parameters for exp:

+----------------+--------------------------------------+
|      name      |                 values               |
+----------------+--------------------------------------+
|    num_ants    |                  int                 |
+----------------+--------------------------------------+
| num_iterations |                  int                 |
+----------------+--------------------------------------+
|  num_clusters  |                  int                 |
+----------------+--------------------------------------+
|  starting_city |                  int                 |
+----------------+--------------------------------------+
|      alpha     |                 float                |
+----------------+--------------------------------------+
|      beta      |                 float                |
+----------------+--------------------------------------+
|        Q       |                  int                 |
+----------------+--------------------------------------+
|        e       |                 float                |
+----------------+--------------------------------------+
|   h_function   |             "Qd" | float             |
+----------------+--------------------------------------+
|    variation   | {                                    |
|                |  approach: "as" | "mmas" | "elitist" |
|                |  upper: float                        |
|                |  lower: float                        |
|                |  type: "global" | "local"            |
|                | }                                    |
+----------------+--------------------------------------+
"""



# ---------------------------------------------------------#

run_experiments(exp)

Starting 10 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/10 [00:00<?, ?engine/s]

create_and_run_ants:   0%|          | 0/10 [00:00<?, ?tasks/s]

Stopping engine(s): 1702414192
engine set stopped 1702414192: {'engines': {'0': {'exit_code': 1, 'pid': 33548, 'identifier': '0'}, '1': {'exit_code': 1, 'pid': 31480, 'identifier': '1'}, '2': {'exit_code': 1, 'pid': 28752, 'identifier': '2'}, '3': {'exit_code': 1, 'pid': 23768, 'identifier': '3'}, '4': {'exit_code': 1, 'pid': 29412, 'identifier': '4'}, '5': {'exit_code': 1, 'pid': 5500, 'identifier': '5'}, '6': {'exit_code': 1, 'pid': 23744, 'identifier': '6'}, '7': {'exit_code': 1, 'pid': 20960, 'identifier': '7'}, '8': {'exit_code': 1, 'pid': 17776, 'identifier': '8'}, '9': {'exit_code': 1, 'pid': 33408, 'identifier': '9'}}, 'exit_code': 1}
Stopping controller
Controller stopped: {'exit_code': 1, 'pid': 26852, 'identifier': 'ipcontroller-1702414191-6dvx-28648'}
Starting 10 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/10 [00:00<?, ?engine/s]

...

  0%|          | 0/10 [00:00<?, ?engine/s]

create_and_run_ants:   0%|          | 0/10 [00:00<?, ?tasks/s]

Stopping engine(s): 1702414543
engine set stopped 1702414543: {'engines': {'0': {'exit_code': 1, 'pid': 33656, 'identifier': '0'}, '1': {'exit_code': 1, 'pid': 27680, 'identifier': '1'}, '2': {'exit_code': 1, 'pid': 34640, 'identifier': '2'}, '3': {'exit_code': 1, 'pid': 32684, 'identifier': '3'}, '4': {'exit_code': 1, 'pid': 30380, 'identifier': '4'}, '5': {'exit_code': 1, 'pid': 26748, 'identifier': '5'}, '6': {'exit_code': 1, 'pid': 13172, 'identifier': '6'}, '7': {'exit_code': 1, 'pid': 33092, 'identifier': '7'}, '8': {'exit_code': 1, 'pid': 3492, 'identifier': '8'}, '9': {'exit_code': 1, 'pid': 34800, 'identifier': '9'}}, 'exit_code': 1}
Stopping controller
Controller stopped: {'exit_code': 1, 'pid': 34592, 'identifier': 'ipcontroller-1702414542-3rxp-28648'}
Starting 10 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/10 [00:00<?, ?engine/s]

create_and_run_ants:   0%|          | 0/10 [00:00<?, ?tasks/s]

## Run Singular ACO

# Unit Tests